In [1]:
import numpy as np
import pandas as pd

# Задание 1

Будем удалять из множества на $n$ элементов рандомный элемент, пока не останется $k$ элементов, то есть нужно удалить $n-k$ элементов. Вероятность получить данное подмножество равно $\dfrac{1}{n}\cdot...\cdot\dfrac{1}{k+1}\cdot(n-k)!$. На файториал домножили, потому что порядок удаления нам не важен. Заметим, что получившееся число равно $\dfrac{1}{C_n^k}$. А это то, что нам надо.

In [2]:
def random_choice(n, k):
    '''
    returns k random indices 
    '''
    indices = np.arange(n)
    for i in range(n - k):
        indices = np.delete(indices, np.random.randint(low=0, high=n - i))
    return indices
print(random_choice(5, 2))

[2 4]


# Задание 2

Рассмотрим один шаг алгоритма. Там мы сравниваем все элементы с опорным элементом. Заметим, что на следующем шаге элементы, которые попали по разные стороны от опорного, больше не сравниваются, то есть любая пара сравнивается не более одного раза. Пусть исходный массив был $A$, $A_{(i)}$ -- $i$-й минимальный по величине элемент, $A_{ij}$ -- элементы от $A_{(i)}$ до $A_{(j)}$. Пусть $S$ -- общее число сравнений, тогда
$$
E[S] = E\big[\sum_{1 \leq i < j \leq n} I\{A_{(i)} \text{ сравнился с } A_{(j)}\}\big] = \sum_{1 \leq i < j \leq n} P\{A_{(i)} \text{ сравнился с } A_{(j)}\}
$$
Ну а когда у нас сравниваются элементы? Очевидно, что когда один из них -- опорный, а так же они должны быть в одном еще не подленном массиве. Вероятность выбрать опорным элементом $A_{(i)}$ равна $\dfrac{1}{j - i + 1}$. Аналогично для $A_{(j)}$. Подставим в сумму:
$$
\sum_{1 \leq i < j \leq n} \dfrac{2}{j - i + 1} = \sum_{i = 1}^{n - 1}\sum_{j = i + 1}^{n} \dfrac{2}{j - i + 1} =
\sum_{i = 1}^{n - 1}\sum_{d = 1}^{n-i} \dfrac{2}{d + 1}
$$
Асимптотически при $n \to \infty$. Это будет не более $O(n\ln n)$, так как сумма по $d$ не более $\ln n$

# Задание 3

__(a)__

Что за событие $\hat\theta > x_{(i)}$? Это значит, что в бутстрепную выборку попало не более 5 элементов из множества $\{x_{(j)}\}_{j=1}^i$. Вероятность такого события определяется выпадением пяти успехов в схеме бернулли с вероятностью выбрать число из множества выше с вероятностью успеха $\dfrac{i}{n}$, где $n=11$ -- размер выборки. Теперь очевидно, почему формула именнто такая

__(б)__

$$
\sum_{j = 0}^{5}\big(Bin(j, n, \frac{i - 1}{n}) - Bin(j, n, \frac{i}{n})\big) = P(\hat\theta > x_{(i-1)}) - P(\hat\theta > x_{(i)}) = 1 - P(\hat\theta \leq x_{(i-1)}) - 1 + P(\hat\theta \leq x_{(i)}) = P(\hat\theta \leq x_{(i)}) - P(\hat\theta \leq x_{(i-1)}) = P\big(\hat\theta \in (x_{(i-1)};x_{(i)}]\big) = P\big(\hat\theta = x_{(i)}\big)
$$

__(c)__

In [3]:
from scipy.stats import binom
n = 11

def p_bigger(i):
    return sum([binom.pmf(j, n, i / n) for j in range(6)])
def p_equal(i):
    return p_bigger(i - 1) - p_bigger(i)
def get_interval(low, high):
    p_theta_bigger_low = p_bigger(low)
    p_theta_eq_low = p_equal(low)
    p_theta_bigger_high = p_bigger(high)
    return p_theta_bigger_low + p_theta_eq_low - p_theta_bigger_high

$P(x_{(3)} \leq \hat\theta \leq x_{(9)}) = 1 - P(\hat\theta < x_{(3)} ) - P(\hat\theta > x_{(9)}) = P(\hat\theta \geq x_{(3)}) - P(\hat\theta > x_{(9)}) = P(\hat\theta > x_{(3)}) + P(\hat\theta = x_{(3)}) - P(\hat\theta > x_{(9)})$

In [4]:
for low in range(1, 12):
    for high in range(low + 1, 12):
        p = get_interval(low, high)
        if p >= 0.89:
            print('[%d, %d] -- %.4f' % (low, high, p))

[1, 8] -- 0.9488
[1, 9] -- 0.9928
[1, 10] -- 0.9998
[1, 11] -- 1.0000
[2, 8] -- 0.9486
[2, 9] -- 0.9926
[2, 10] -- 0.9997
[2, 11] -- 0.9998
[3, 8] -- 0.9415
[3, 9] -- 0.9856
[3, 10] -- 0.9926
[3, 11] -- 0.9928
[4, 8] -- 0.8975
[4, 9] -- 0.9415
[4, 10] -- 0.9486
[4, 11] -- 0.9488


Самый нормальный интервал(в википедии почему-то с равенством, то есть это отрезок) -- $[x_{(3)}, x_{(8)}]$ и $[x_{(4)}, x_{(9)}]$ с уровнем доверия $0.94$

Хотя если округлять до второго знака, то еще будет $[x_{(4)}, x_{(8)}]$ с уровнем доверия $0.8975 \approx 0.9$

# Задание 4

In [92]:
! wget -nc https://vincentarelbundock.github.io/Rdatasets/csv/MASS/galaxies.csv -P ./data/

File ‘./data/galaxies.csv’ already there; not retrieving.



In [93]:
df = pd.read_csv('data/galaxies.csv', index_col=0)
df[:5]

,dat
1,9172
2,9350
3,9483
4,9558
5,9775


__(a)__

Будем использовать Гауссово ядро как в Википедии, то есть $\dfrac{1}{2\pi}exp(\dfrac{x^2}{2})$, потому что в чате пошла путанница с нормировкой.

In [98]:
from tqdm import tqdm
def get_min_h(x_from, x_to, x_count, data):
    x_arr = np.linspace(x_from, x_to, x_count)

    def get_density(x, h):
        after_kernel = np.exp( -((x - data) / h)**2 / 2 ) / np.sqrt(2 * np.pi)
        return after_kernel.sum() / data.shape[0] / h 

    def is_unimodal(h):
        assert(data.shape[0] > 2)
        density = np.array([get_density(x, h) for x in x_arr])
        counter = 0
        for i in range(len(density) - 2):
            last_d, cur_d, next_d = density[i - 1], density[i], density[i + 1]
            if last_d < cur_d and cur_d > next_d:
                counter += 1
            last_d = cur_d
            cur_d = next_d
        return counter == 1
    
    h_left = 1e-4
    h_right = 1e4
    h_min = 0
    while abs(h_left - h_right) > 0.01:
        h_min = (h_left + h_right) / 2
        if is_unimodal(h_min):
            h_right = h_min
        else:
            h_left = h_min
    return h_min

In [104]:
h_uni = get_min_h(x_from=5000, x_to=50000, x_count=1000, data=df.values)
print(h_uni)

3045.835564536761


__(б)__

In [111]:
from sklearn.utils import resample
def get_h(data):
    return get_min_h(x_from=5000, x_to=50000, x_count=1000, data=data)

In [114]:
B = df.shape[0]
indic = 0
for b in tqdm(range(B)):
    data = resample(df.values)
    indic += get_h(data) >= h_uni
p = indic / B
print(p)

100%|██████████| 82/82 [00:41<00:00,  2.00it/s]

0.4634146341463415


# Задание 5